Code pour projet crypto

Import des librairies

**Date limite :** Mercredi 27 mars 2024

**Groupe TD DIA 6 :**

Henri Serano, Sara Thibierge, Eloi Seidlitz

In [14]:
import hashlib
import random 
import gmpy2 as gm
import math
import time
import base64
import os, binascii
from backports.pbkdf2 import pbkdf2_hmac
import pyaes, pbkdf2, secrets


## Exercice 1

1. Trouver une chaîne de caractères (contenant vos noms et prénoms) dont le hash SHA256 se termine par le plus de zéros possible (en hexadécimal).

2. Mesurer le temps moyen pour obtenir n et n+1 zéros en fin de chaîne (n = 5) et calculer le rapport Tn+1/Tn.

### Partie 1

In [15]:

def trouver_chaine(base, zeros_cibles):
        compteur = 0
        while True:
            chaine_test = base + str(compteur)
            hash_resultat = sha256_hash(chaine_test)
            if hash_resultat.endswith('0' * zeros_cibles):
                return chaine_test, compteur, hash_resultat
            compteur += 1

def sha256_hash(chaine):
        return hashlib.sha256(chaine.encode()).hexdigest()
        
def trouver_chaine_et_stats(base, zeros_cibles, iterations_stat=5):
    

    # Première partie : Trouver la chaîne
    chaine_trouvee, compteur_trouve, hash_trouve = trouver_chaine(base, zeros_cibles)

    # Deuxième partie : Statistiques de temps
    temps_n = []
    temps_n_plus_1 = []
    
    for _ in range(iterations_stat):
        debut = time.time()
        trouver_chaine(base, zeros_cibles)
        fin = time.time()
        temps_n.append(fin - debut)

        debut = time.time()
        trouver_chaine(base, zeros_cibles + 1)
        fin = time.time()
        temps_n_plus_1.append(fin - debut)

    temps_moyen_n = sum(temps_n) / len(temps_n)
    temps_moyen_n_plus_1 = sum(temps_n_plus_1) / len(temps_n_plus_1)

    rapport_temps = temps_moyen_n_plus_1 / temps_moyen_n

    return {
        "chaine_trouvee": chaine_trouvee,
        "hash_trouve": hash_trouve,
        "compteur_trouve": compteur_trouve,
        "temps_moyen_n": temps_moyen_n,
        "temps_moyen_n_plus_1": temps_moyen_n_plus_1,
        "rapport_temps": rapport_temps
    }


base_chaine = "Serano_Henri__Sara_Thibierge__Eloi_Seidlitz_DIA6"
zeros_cibles = 4 # Nombre de zéros à la fin du hash
resultats = trouver_chaine_et_stats(base_chaine, zeros_cibles)
resultats


{'chaine_trouvee': 'Serano_Henri__Sara_Thibierge__Eloi_Seidlitz_DIA677031',
 'hash_trouve': '31e6f8e6377ad1c9895ef7c22b0ef9a252c0f0ec9d062178c58b416bcb470000',
 'compteur_trouve': 77031,
 'temps_moyen_n': 0.13479971885681152,
 'temps_moyen_n_plus_1': 0.3936333179473877,
 'rapport_temps': 2.9201345617457655}

## Exercice 2

1. Chiffrer un texte d'environ une demi-page (format .txt) avec AES256-CTR-PBKDF2 en utilisant un mot de passe et une valeur de compteur (IV) de 6 à 9 chiffres (sans 0).

2. Constituez un entier N en accolant le mot de passe et l'IV séparés par quatre 0.

3. Chiffrer N avec ElGamal en utilisant les nombres p, g et A fournis.

4. Sauvegarder le fichier chiffré avec l'extension .enc.

In [16]:
with open("format.txt", "r",encoding="utf-8") as file:
    text = str(file.read())
text

"Le marché des sneakers en France a connu une croissance exponentielle au cours des dernières années, devenant un phénomène de mode et un investissement lucratif pour de nombreux amateurs et professionnels. \nCe texte explore les multiples faceteurs qui sous-tendent le commerce de sneakers dans l'Hexagone, en mettant en lumière ses dynamiques économiques, culturelles et sociales.\nD'un point de vue économique, le commerce de sneakers en France est stimulé par une demande croissante pour des modèles rares et exclusifs. \nLes éditions limitées et les collaborations entre marques de renom et designers célèbres créent une urgence d'achat, poussant les prix sur le marché secondaire à atteindre des sommets vertigineux. \nCette situation a engendré l'émergence d'une véritable économie parallèle, où les sneakers sont non seulement des objets de mode, mais également des actifs spéculatifs.\n\nSur le plan culturel, les sneakers occupent une place prépondérante dans l'univers de la mode et de la 

In [17]:
password = "123456789"
passwordSalt = os.urandom(16)
key = pbkdf2.PBKDF2(password, passwordSalt).read(32)
iv = secrets.randbits(256)
#On est obliger de recrée un mot de pass unique pour chiffrer le texte
aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))
ciphertext = aes.encrypt(text)

In [18]:
print(aes.decrypt(ciphertext))

b'\xca\xd0\xe5\xa3I\xd7\xd1}=\x85\x85\xdc\\\xd3`\xac(1"3^\xddh\xfc\xa9\xd3\x94\xb5\xcb\x8c\xb8\xb6\xff\xe3\xdeO\xd8pv9\xba\x00\xab\xa8e\xe0rS\x12\xf7\x87 \x9ch]\xd2V\x889\x80\xb4y\xf1\xfc\xee\xeb\x9cL:\x81\xdd\x14P\x82\xc7z6\xbc9\xc2\xe7\xc5\xe7\xfe\x02&\xf0\xa8\x0b8\xe6\xcd k\x8b^\xd7R\xf7\xff!RuL\x1f\xac\xab\xac\xf6\xab\x92\xb5s\x07\xc9\x1f\xfc\xf14\x99)\x1a\x01\x8bk\x1d\xe2\x19\x07\x96\xc8\x9e*\xa8\xd40f\xb5\xae\xba\xd4\x05\x1e\x12\noAn\x98\x0f\xbbS\x17\xc5\xe8\xacW\xc516t\xf0\x9d\xd9\xd8\x1fg`\xf1\x94\x7f\xb6o\x82T\xa4\x19\xa7I\xd9Y\xd1\xa2\x04\x89\x9aj\x84\xa8\xfd^\x82\xa8\xee\xe2\xaf}\xba\xeedd\x98\x9d\xa4\xcc\x94\x86\xae"\xfehYm\\\xac\xeb\xa9\x0e\xcd\x17\x96M\xbe\xdcP\xd8@b\xa1\x93\x1cA^\xdf\x05\xbd\x15j\xbc\x11\xe0_\xe3\xad\xe0<\x9cLi\xd1dw7\xfc^!\x1e\xee\x14\xbf\xf1\x87\xd6\xbf\xd2\xc8\'\x8c\xa1bz\x06b\xfa\x1f\xa7}\xfd\xfb\x06U+*S\xb7\xf9d\xc0\xda\xc7\x9aNL\x99z\x90\x9c\xa4N\x06>p\xad<\xe0\xa5\xd0\xd4I\x90\xe7\x9a\xd1\xe3;\x17\x0e6\xca\xb3\x1f\t\xda 8}\x1b\xb9J\xc62v\x96\xac\x

In [19]:
print(pyaes.Counter(iv))
#Et du coup de le regénéré
aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))
print(aes.decrypt(ciphertext))

b"Le march\xe9 des sneakers en France a connu une croissance exponentielle au cours des derni\xe8res ann\xe9es, devenant un ph\xe9nom\xe8ne de mode et un investissement lucratif pour de nombreux amateurs et professionnels. \nCe texte explore les multiples faceteurs qui sous-tendent le commerce de sneakers dans l'Hexagone, en mettant en lumi\xe8re ses dynamiques \xe9conomiques, culturelles et sociales.\nD'un point de vue \xe9conomique, le commerce de sneakers en France est stimul\xe9 par une demande croissante pour des mod\xe8les rares et exclusifs. \nLes \xe9ditions limit\xe9es et les collaborations entre marques de renom et designers c\xe9l\xe8bres cr\xe9ent une urgence d'achat, poussant les prix sur le march\xe9 secondaire \xe0 atteindre des sommets vertigineux. \nCette situation a engendr\xe9 l'\xe9mergence d'une v\xe9ritable \xe9conomie parall\xe8le, o\xf9 les sneakers sont non seulement des objets de mode, mais \xe9galement des actifs sp\xe9culatifs.\n\nSur le plan culturel, les s

In [20]:
p = 7946851324679854613245823
g = 5
A = 7579501795988122393422986
## Elle publie A
password = "35426517"
IV = "756431" 
N = int(password + "0000" + IV) 

b = 123456789  

# Calcul de B et C pour ElGamal
B = pow(g, b, p)
K = pow(A, b, p)  # Clé secrète partagée
C = (K * N) % p  # Chiffrement de N avec la clé secrète partagée

B, C

(387225946877039798412155, 5883019579684883513940408)

## Exercice 3

* Démonstration de la signature ElGamal ou RSA (avec des nombres différents de ceux du cours).
* Exemple d'utilisation du RSA, Diffie-Hellman, ElGamal ou de la signature ElGamal dans Python ou openssl (contexte réel).

#### 1. Démonstration de la signature ElGamal ou RSA (avec des nombres différents de ceux du cours).

In [21]:
# Fonction pour calculer l'inverse modulo
def inverse_modulo(a, m):
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1

# Fonction pour calculer la signature
def sign_elgamal(m, x, p, g):
    # Sélection d'un nombre aléatoire k
    k = random.randint(2, p - 2)
    while pow(k, -1, p - 1) == 0:  # Assure que k est premier avec p-1
        k = random.randint(2, p - 2)
    r = pow(g, k, p)
    k_inv = inverse_modulo(k, p - 1)
    s = (m - x * r) * k_inv % (p - 1)
    return r, s

# Fonction pour vérifier la signature
def verify_elgamal(m, r, s, y, p, g):
    v1 = pow(g, m, p)
    v2 = (pow(y, r, p) * pow(r, s, p)) % p
    return v1 == v2

# Paramètres ElGamal
p = 257
g = 3
x = 97

# Clé publique
y = pow(g, x, p)

# Message à signer
m = 123

# Signature
r, s = sign_elgamal(m, x, p, g)

# Vérification de la signature
valid = verify_elgamal(m, r, s, y, p, g)

print("Signature valide :", valid)


ValueError: base is not invertible for the given modulus

#### 2. Exemple d'utilisation du RSA, Diffie-Hellman, ElGamal ou de la signature ElGamal dans Python ou openssl (contexte réel).

Supposons que nous ayons un fichier texte appelé "format.txt" contenant le message à signer.



**a) Génération des clés RSA :**
Tout d'abord, nous devons générer une paire de clés RSA privée/publique à l'aide d'OpenSSL.

In [ ]:
#copy paste in a terminal
!openssl genpkey -algorithm RSA -out private_key.pem
!openssl rsa -pubout -in private_key.pem -out public_key.pem

.........+....+.....+...............+.+..+++++++++++++++++++++++++++++++++++++++*.....+...+......+.+........+.......+..............+....+..+.+..+.+......+...+......+..+...+.......+.....+.......+..+++++++++++++++++++++++++++++++++++++++*.+..+...................+......+........+......+.+........+...+....+......+.........+......+.....+...+....+...+.....+...+.......+........+....+............+...+..................+...+..+.......+..+.+...+.....+................+...+.....+......+.+...+..+................+...+..+....+........+.+.........+........+....+.....+...+.+...........................+.....+...+......+......+.......+......+......+.....+...+.+.........+........+...+......++++++
.+...........+...+.+......+...+..+...+.......+......+.....+++++++++++++++++++++++++++++++++++++++*........+++++++++++++++++++++++++++++++++++++++*....+....+.....+.+............+..+......+......+.+..+...+....+......+.....+....+.........+..+...+......+....+..+...............+......................+..+..............

Ces commandes génèrent une clé privée private_key.pem et une clé publique correspondante `public_key.pem`.



**b) Signature du message :**

Ensuite, nous signons le message avec notre clé privée.

In [ ]:
#copy paste in a terminal
!openssl dgst -sha256 -sign private_key.pem -out signature.bin format.txt

Cela crée une signature du message dans le fichier signature.bin.


**c) Vérification de la signature :**

Pour vérifier la signature avec la clé publique correspondante, nous utilisons :

In [ ]:
#copy paste in a terminal
!openssl dgst -sha256 -verify public_key.pem -signature signature.bin format.txt

Verified OK


Si la vérification réussit, OpenSSL affichera "Verified OK".


Cet exemple montre comment utiliser OpenSSL pour générer une paire de clés RSA, signer un message avec la clé privée et vérifier la signature avec la clé publique. Vous pouvez répéter ces étapes dans un terminal pour expérimenter avec vos propres messages et clés.

In [ ]:
!openssl genpkey -algorithm RSA -out private_key.pem
!openssl rsa -pubout -in private_key.pem -out public_key.pem
!openssl dgst -sha256 -sign private_key.pem -out signature.bin format.txt       
!openssl dgst -sha256 -verify public_key.pem -signature signature.bin format.txt

.....+++++++++++++++++++++++++++++++++++++++*.....+...+......+++++++++++++++++++++++++++++++++++++++*...+.+........+.......+..+.......+.........+.....+......+.+..+.+.........+.....+......+....+...........+......+....+.....+......+.........+.+......+.....+....+...+...+..............+.+.....+....+..................+...+........+......+...+...............+............+.+..+.......++++++
......+...+...........+...+.+++++++++++++++++++++++++++++++++++++++*.+.+.................+..........+..............+.+..+...+.......+..+...+.+..+++++++++++++++++++++++++++++++++++++++*..+....+......+..+......+.......+..+...+...+...................+...............+..+.+............+........+...++++++


writing RSA key


Verified OK
